# Notebook de preprocesamiento para el experimento

El objetivo de este experimento es lanzar un entrenamiento federado para lograr un modelo entrenado con tres datasets aislados. En usuario tendra uno de ellos. El objetivo es comunicarse con los otros 3 compañeros en como establecer un dataset con una estructura común, y lanzar el entrenamiento. Para ello, se espera que los usuarios de este notebook realicen lo siguiente:


- Construir el dataset con la misma estructura o proponer otra versión del dataset. Para construir el dataset:
    - Ordenar las columnas del dataset en el mismo orden del dataset definido en la plataforma.
    - Eliminar las columnas que puedan sobrar, o crear aquellas que falten.
        - Si va a crear columnas, es recomendable rellenarlas con un único valor, para no introducir ruido en el dataset.
- Preprocesar el dataset para:
    - Escalar los valores numericos a la misma escala que los otros participantes.
    - Procesar los valores categoricos para transformarlos en valores numericos, ya sea en forma de lista, o de one-hot-encoded (una columna por valor, todas a 0 excepto aquella que correspondia al valor original, que se establece a 1).

En esta parte del código, se espera que el alumno preprocese los datos para el entrenamiento. Debido a que la información que cada silo (participante dentro del grupo) contiene es diferente, es necesario que intercambieis cierta información antes de procesar los datos, para no generar inconsistencias.

Para la parte de la plataforma, solo es necesario modificar el dataset a nivel de crear o eliminar columnas. El preprocesamiento es realizado por la plataforma.

**Nota: Las clases (label) ya se han proporcionado en la parte superior del notebook.**

## Petición: Mejor no tocar el codigo hasta que no se indique. Por temas de consistencia.

In [1]:
import pandas as pd

In [2]:
name_dataset = "wine_dataset_1"

In [7]:
dataset = pd.read_csv(f"data/raw_data/{name_dataset}.csv", index_col=0)
y = dataset.pop("label")
# y = y.apply(str)
X = dataset.copy()

### Preprocessing

In the following block, we provide with an example on how to call the functions for preprocessing the datasets. Each preprocessing function is meant to preprocess a type of data, either numerical or categorical, and it requires the name of the column, as well as some additional information. The list is the following:
- *Min max scaler* (numérico): Toda la información se reescala en base al valor mínimo y máximo de la columna, pasando el valor máximo a ser 1, y el mínimo 0.
    - Ejemplo: [0, 1, 2] -> [0, 0.5, 1]
    - Input: Column (X), minimum global value, maximum global value.
- *Label Encoder* (categórico): A cada valor categórico se le asigna un valor númerico equivalente, partiendo de 0 hasta considerar todas las categorías. Importante: Este encoder genera una lista que se puede interpretar como un order jerarquico.
    - Ejemplo: [rojo, azul, verde] -> [0, 1, 2]
    - Input: Columna (X), lista con todas las categorias.
- One hot encoder (categórico): Por cada valor categórico se genera un vector de N dimensiones, siendo N el número de categorias que se encuentran en la columna. El vector tiene todos los valores a 0 exceptuando el de la categoría original, que se encuentra a 1.
    - Ejemplo: [rojo, azul, verde] -> [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    - Input: Columna, lista con todas las categorias.

**Nota: Aunque en las siguientes funciones tienen como parametro de entrada la X, dicha X representa una columna. La y (la etiqueta de objetivo) tambien puede considerarse una columna, y procesarse con estas funciones.**

In [ ]:
def min_max_scaler(X, global_min, global_max):
    X = (X - global_min) / (global_max - global_min)
    return X

In [ ]:
def one_hot_encode_column(X, categories):
    encoded_dataframe = pd.DataFrame()
    X = X.str.strip()
    X = X.str.lower()
    for category in categories:
        category = category.strip()
        category = category.lower()
        encoded_dataframe[category] = X == category
        encoded_dataframe[category] = encoded_dataframe[category].astype(int)
    # encoded_dataframe[X.loc[:]] = 1
    return encoded_dataframe

In [ ]:
def label_encoder(X, categories):
    map_categories = {category: number for category, number in zip(categories, range(len(categories)))}
    return X.map(map_categories)

Para usar las funciones de preprocesamiento, tenemos que usar la columna del dataset, y pasarla a la función. Abajo puede encontrar un ejemplo para cada una de las funciones:

In [ ]:
# Before preprocessing
display(X["alcohol"].head())

# Min Max Scaler
display(min_max_scaler(X["alcohol"], 10, 20).head())

### Funciones adicionales de preprocesamiento

Ordenar columnas: Para ordenar columnas del dataframe o dataset, simplemente seleccione las columnas en el orden necesario. Para ello, seleccione las columnas de las siguiente forma: X[[primera_columna, segunda_columna, ...]].

Eliminar columnas: Para ello, simplemente no la incluya en la lista para ordenar.

Crear columnas: Para crear una columna, simplemente seleccione el dataset y la columna, e introduzca el valor de la siguiente forma:
X["columna"] = valor

Cambiar valores de categorias en la columna:
X["columna"] = X["columna"].map{"categoria_original_1": "categoria_a_cambiar_1", "categoria_original_2": "categoria_a_cambiar_2", ...}


## En esta sección es donde se espera que escribais algo de código. Abra una indicación al final de la misma.

En esta parte del código, se espera que el alumno preprocese los datos para el entrenamiento. Debido a que la información que cada silo (participante dentro del grupo) contiene es diferente, es necesario que intercambieis cierta información antes de procesar los datos, para no generar inconsistencias.

La información sería:
- Nombres de las columnas, y orden. Nombres de las columnas, y orden. Importante que sea el mismo para todos los participantes.
- Valores máximos y mínimos de las columnas númericas (si min max encoder). Formato: Dos valores númericos por separado.
- Valores categóricos de las columnas categóricas (si label encoder o one hot encoder). Formato: Es una lista.

**Nota: Las clases (label) ya se ha proporcionado en la parte superior del notebook.**

## Final de la sección de código a modificar.

## A partir de aquí, mejor no tocar. Simplemente ejecute uno a uno los bloques de código.

Para realizar una tarea de clasificación, es necesario que usemos el one-hot-encoder en la columna de la etiqueta (label).

In [ ]:
y_one_hot_encoded: pd.DataFrame = one_hot_encode_column(y, ["red", "white", "sparkling"])
y_one_hot_sorted = y_one_hot_encoded.sort_index(axis=1)

Aquí se realiza la división de datos entre entrenamiento y test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_sorted, random_state=1, test_size=0.3)

Finalmente, se almacenan dichos datos para el entrenamiento.

In [ ]:
import os

os.makedirs(f"data/training_data_preprocessed/{name_dataset}", exist_ok=True)
X_train.to_csv(f"data/training_data_preprocessed/{name_dataset}/" + "X_train.csv")
X_test.to_csv(f"data/training_data_preprocessed/{name_dataset}/" + "X_test.csv")
y_train.to_csv(f"data/training_data_preprocessed/{name_dataset}/" + "y_train.csv")
y_test.to_csv(f"data/training_data_preprocessed/{name_dataset}/" + "y_test.csv")